## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [6]:
# requests 라이브러리 설치여부 확인
!pip show requests

Name: requests
Version: 2.32.3
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: anaconda-catalogs, anaconda-client, anaconda-cloud-auth, anaconda-project, conda, conda-build, conda-repo-cli, conda_package_streaming, cookiecutter, datashader, jupyterlab_server, panel, requests-file, requests-toolbelt, Sphinx, streamlit, tldextract


In [7]:
# beautifulsoup4 라이브러리 설치여부 확인
!pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.12.3
Summary: Screen-scraping library
Home-page: https://www.crummy.com/software/BeautifulSoup/bs4/
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: soupsieve
Required-by: conda-build, nbconvert


In [8]:
# reqeusts, bs4 import
import requests
import bs4
# bs4 모듈 속 BeautifulSoup 클래스 import
from bs4 import BeautifulSoup

In [9]:
# requests, bs4 버전 확인하기
print(f'requests 버전 = {requests.__version__}')
print(f'bs4 버전 = {bs4.__version__}')

requests 버전 = 2.32.3
bs4 버전 = 4.12.3


### 1. 뉴스 제목 추출하기

In [10]:
# IT/과학 뉴스 
req_param = {
    'sid': 105
}

sid = 104
# url = f'https://news.naver.com/section/{sid}'
url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)

https://news.naver.com/section/105


In [11]:
req_param = {
    'sid': 105
}

url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)

# 요청 헤더 설정 : 브라우저 정보
req_header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

# requests 의 get() 함수 호출하기 
res = requests.get(url, headers=req_header)
# 응답(response)이 OK 이면 200 출력
print(res.status_code, res.ok) # 응답 상태 코드 출력
print(type(res))
print(res)

# 응답 (response)에서 text 추출
if res.ok:
    html = res.text
    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(html, 'html.parser')
    # print(soup)
    # CSS 선택자
    # print(soup.select("div.sa_text a[href*='mnews/article']"))
    '''
        <div class="sa_text">
            <a href="https://n.news.naver.com/mnews/article/366/0001094461" class="sa_text_title">
    '''
    a_tags = soup.select("div.sa_text a[href*='mnews/article']") # div class 속성은 .
    print(len(a_tags))
    print(type(a_tags)) # [Tag, Tag, Tag]

    # <a> 태그 리스트 순회하기
    for idx, a_tag in enumerate(a_tags, 1):
        if a_tag.text:
            title = a_tag.text.strip() # 제목
            link = a_tag['href'] # 링크
            # print(len(title))
            print(f'{idx} {title} {link}')
else:
    # 응답(response)이 Error 이면 status code 출력
    print(f'Error Code: {res.status_code}')

https://news.naver.com/section/105
200 True
<class 'requests.models.Response'>
<Response [200]>
88
<class 'bs4.element.ResultSet'>
1 100만원짜리 폰이 공짜?… 단통법 폐지, 시작된 ‘보조금 전쟁’ https://n.news.naver.com/mnews/article/023/0003918587
3 LG U+, '갤럭시 Z 폴드7·플립7' 사전예약 고객 개통 시작 https://n.news.naver.com/mnews/article/014/0005380387
5 외국인 고객 공략 나선 LGU+…로밍·데이터 혜택 강화 https://n.news.naver.com/mnews/article/277/0005626573
7 美 AI 인재 유입 비상… 해외 인재 7% 의존 속 '탈(脫)미국' 가속화 https://n.news.naver.com/mnews/article/018/0006071776
9 삼성, 갤워치 울트라 등 과거 모델에도 '원 UI 8 워치' 업데이트 https://n.news.naver.com/mnews/article/003/0013379022
11 크라우드웍스, 'K-AI' 구축 사업에 KT 컨소시엄으로 참여 https://n.news.naver.com/mnews/article/003/0013378971
13 쏘카 타고 많이 간 여름 해수욕장은?…1·2위 모두 '부산' https://n.news.naver.com/mnews/article/421/0008386525
15 CJ ENM, 韓 엔터사 최초 사우디에 법인 설립 https://n.news.naver.com/mnews/article/009/0005529568
17 KT, 독자 AI 파운데이션 모델 개발 출사표 https://n.news.naver.com/mnews/article/092/0002383131
19 [단독] 웹툰 '유일무이 로맨스' 영상화…네이버웹툰 IP 넓어진다 https://n.ne

### 1.1 뉴스제목 추출하는 함수 선언하기

In [12]:
import requests
from bs4 import BeautifulSoup

#section_dict = {100:'정치',101:'경제',102:'사회',103:'생활/문화',104:'세계',105:'IT/과학'}
section_dict = {'정치':100,'경제':101,'사회':102,'생활/문화':103,'세계':104,'IT/과학':105}

def print_news(section):  #print_new(103,'생활/문화') 
    sid = section_dict[section]
    url = f'https://news.naver.com/section/{sid}'
    print(f'{url} {section} 뉴스')

    # 요청 헤더 설정 : 브라우저 정보
    req_header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }
    # requests 의 get() 함수 호출하기 
    res=requests.get(url,headers=req_header)
    print(res.status_code, res.ok)
    if res.ok:
        html = res.text
        # BeautifulSoup 객체 생성
        soup = BeautifulSoup(html, "html.parser")
        a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/article'].sa_text_title")
        print(len(a_tags))
        # <a> 태그 리스트 순회하기
        for idx,a_tag in enumerate(a_tags,1):
            if a_tag.text:
                title = a_tag.text.strip() # 제목
                link = a_tag['href'] # 링크
                #print(len(title))
                print(f'{idx} {title} {link}')
    else:
        # 응답(response)이 Error 이면 status code 출력    
        print(f'Error Code = {res.status_code}')

In [13]:
# print_news(103,section_dict[103])
print_news('세계')

https://news.naver.com/section/104 세계 뉴스
200 True
46
1 미 국방 “필리핀과 상호방위조약, 태평양 전체 적용”…한국에도 영향? https://n.news.naver.com/mnews/article/028/0002757347
2 방글라데시 훈련기 학교 추락 사고 사망자 31명으로 늘어 https://n.news.naver.com/mnews/article/277/0005626456
3 트럼프 "日과 관세 15%에 무역 합의…車·쌀 시장 개방" https://n.news.naver.com/mnews/article/123/0002364259
4 "유치한 물타기" 오바마, 트럼프 "반역" 주장에 정면 반박 https://n.news.naver.com/mnews/article/003/0013378957
5 벼랑끝 또 만난 원수… 아소, 이시바 퇴진 압박 https://n.news.naver.com/mnews/article/021/0002724372
6 “日 여행간 딸, 입원했대서 돈 보냈는데”…1개월째 행방불명 https://n.news.naver.com/mnews/article/009/0005528814
7 크렘린궁 "우크라 협상서 어려운 주제 논의…돌파구 기대 안 해" https://n.news.naver.com/mnews/article/003/0013378738
8 이스라엘, 가자 중부까지 첫 지상작전 돌입…공습·폭격 병행 https://n.news.naver.com/mnews/article/421/0008384111
9 다음주 미중 고위급 무역회담…"관세 인하 추가연장 가능성" https://n.news.naver.com/mnews/article/437/0000449690
10 GM, 2분기 매출 및 순이익 하락 불구 예상치 초과 https://n.news.naver.com/mnews/article/003/0013378631
11 엔화, 일본에 15% 상호관세 부과에 1달러=146엔대 후반 상승 출발 https://n.ne

### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [14]:
import requests
import os

req_header = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=840954&no=1&week=tue',
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

img_urls = [
    'https://image-comic.pstatic.net/webtoon/840954/1/20250721113606_15ea4b1dc21b833f3817bd1194f267fb_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/840954/1/20250721113606_15ea4b1dc21b833f3817bd1194f267fb_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/840954/1/20250721113606_15ea4b1dc21b833f3817bd1194f267fb_IMAG01_3.jpg'
]

for img_url in img_urls:
    # requests 의 get(url, headers) 함수 호출하기
    res = requests.get(img_url, headers=req_header)
    print(res.ok, res.status_code)
    if res.ok:
        # binary 응답 데이터 가져오기
        img_data = res.content
        # print(img_data)
        # url에서 파일명만 추출하기
        file_name = os.path.basename(img_url)
        print(file_name)
        # binday data를 file에 write하기
        with open(file_name, 'wb') as file: # wb: write binary
            print(f'Writing {file_name}({len(img_data):,} bytes)')
            file.write(img_data)

True 200
20250721113606_15ea4b1dc21b833f3817bd1194f267fb_IMAG01_1.jpg
Writing 20250721113606_15ea4b1dc21b833f3817bd1194f267fb_IMAG01_1.jpg(135,959 bytes)
True 200
20250721113606_15ea4b1dc21b833f3817bd1194f267fb_IMAG01_2.jpg
Writing 20250721113606_15ea4b1dc21b833f3817bd1194f267fb_IMAG01_2.jpg(252,452 bytes)
True 200
20250721113606_15ea4b1dc21b833f3817bd1194f267fb_IMAG01_3.jpg
Writing 20250721113606_15ea4b1dc21b833f3817bd1194f267fb_IMAG01_3.jpg(370,695 bytes)


* 현재 요청된 페이지의 image 모두 다운로드 해보기

In [ ]:
import requests
from bs4 import BeautifulSoup 
import os

webtoon_url = 'https://comic.naver.com/webtoon/detail?titleId=826887&no=57&week=tue'

req_header = {
    'referer': webtoon_url,
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

res = requests.get(webtoon_url)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select("img[src$='.jpg']")))
    # https://image-comic.pstatic.net/webtoon
    print(len(soup.select("img[src^='https://image-comic.pstatic.net/webtoon/826887/57/20250616174232_f57726d2b95899b88de4b0a4522803be_IMAG01']"))) # 일치
    print(len(soup.select("img[src*='IMAG01']"))) # 일치
    # img src file명을 저장할 list 선언
    img_url_list = list() # ['http://xxx/aa.jpg]
    img_tags = soup.select("img[src*='IMAG01']")
    for img_tag in img_tags: #img_tag 의 타입은 Tag <img>
        img_url = img_tag['src']
        # print(img_url)
        img_url_list.append(img_url)
    # print(len(img_url_list), img_url_list[0:2])

    # img 디렉토리가 없으면 생성하기
    imgdir_name = 'img'
    if not os.path.isdir(imgdir_name):
        os.mkdir(imgdir_name)
    
    for jpg_url in img_url_list:
        res = requests.get(jpg_url, headers=req_header)
        if res.ok:
            # binary data 추출하기
            img_data = res.content
            # img/xxx.jpg 디렉토리명 jpg 파일명을 join
            dir_name = os.path.join(imgdir_name, os.path.basename(jpg_url))
            print(dir_name)
            with open(dir_name, 'wb') as file:
                print(f'Writing {os.path.basename(jpg_url)}({len(img_data):,} bytes)')
                file.write(img_data)

        else:
            print(f'Error Code: {res.status_code}')

### 리펙토링 코드

In [26]:
import requests
from bs4 import BeautifulSoup
import os

# 기본 설정
url = 'https://comic.naver.com/webtoon/detail?titleId=833255&no=3&week=tue'
req_header = {'referer': url}
imgdir_name = 'img'

# 이미지 저장 폴더가 없으면 생성
os.makedirs(imgdir_name, exist_ok=True)

# 웹 페이지 요청 및 확인
res = requests.get(url)
if not res.ok: # 여기서 먼저 거르고 하면 더 깔끔
    print(f'Error Code = {res.status_code}')
    exit()

# 이미지 URL 추출
soup = BeautifulSoup(res.text, 'html.parser')
img_url_list = [img_tag['src'] for img_tag in soup.select("img[src*='IMAG01']")]

# 이미지 다운로드
for img_url in img_url_list:
    res = requests.get(img_url, headers=req_header)
    if res.ok:
        img_data = res.content
        file_path = os.path.join(imgdir_name, os.path.basename(img_url))
        with open(file_path, 'wb') as file:
            print(f'Writing to {file_path} ({len(img_data):,} bytes)')
            file.write(img_data)
    else:
        print(f'Error Code = {res.status_code} for {img_url}')

Writing to img\20241019203626_ee2218f8c6131e687ade8a9df8c9d052_IMAG01_1.jpg (164,266 bytes)
Writing to img\20241019203626_ee2218f8c6131e687ade8a9df8c9d052_IMAG01_2.jpg (119,906 bytes)
Writing to img\20241019203626_ee2218f8c6131e687ade8a9df8c9d052_IMAG01_3.jpg (130,933 bytes)
Writing to img\20241019203626_ee2218f8c6131e687ade8a9df8c9d052_IMAG01_4.jpg (130,725 bytes)
Writing to img\20241019203626_ee2218f8c6131e687ade8a9df8c9d052_IMAG01_5.jpg (134,506 bytes)
Writing to img\20241019203626_ee2218f8c6131e687ade8a9df8c9d052_IMAG01_6.jpg (166,312 bytes)
Writing to img\20241019203626_ee2218f8c6131e687ade8a9df8c9d052_IMAG01_7.jpg (135,069 bytes)
Writing to img\20241019203626_ee2218f8c6131e687ade8a9df8c9d052_IMAG01_8.jpg (95,346 bytes)
Writing to img\20241019203626_ee2218f8c6131e687ade8a9df8c9d052_IMAG01_9.jpg (160,655 bytes)
Writing to img\20241019203626_ee2218f8c6131e687ade8a9df8c9d052_IMAG01_10.jpg (157,106 bytes)
Writing to img\20241019203626_ee2218f8c6131e687ade8a9df8c9d052_IMAG01_11.jpg (15

### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

In [31]:
import requests

upload_files = {
    'img1': open('f1.jpg', 'rb'),
    'img2': open('f2.jpg', 'rb')
}
print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.
res = requests.post(url, files=upload_files)
print(res.status_code)
print(type(res.json()))
# print(res.json())
print(res.json()['files']['img1'])


{'img1': <_io.BufferedReader name='f1.jpg'>, 'img2': <_io.BufferedReader name='f2.jpg'>}
200
<class 'dict'>
...


### 4. 캡차(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [15]:
# 캡차 키 발급 요청
import urllib.request

client_id = "DkiHrMnuqr5Rs9qsPs2f" # 개발자센터에서 발급받은 Client ID 값
client_secret = "LnZR3PAbKk" # 개발자센터에서 발급받은 Client Secret 값

code = "0"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code
# Request 객체 생성
request = urllib.request.Request(url)
# 요청 헤더 설정
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
# 서버에 request 전달
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{"key":"QKVQYCRj1xWmg1MC"}


In [16]:
# 캡차 이미지 요청
import urllib.request

client_id = "DkiHrMnuqr5Rs9qsPs2f" # 개발자센터에서 발급받은 Client ID 값
client_secret = "LnZR3PAbKk" # 개발자센터에서 발급받은 Client Secret 값
key = "QKVQYCRj1xWmg1MC" # 캡차 Key 값
url = "https://openapi.naver.com/v1/captcha/ncaptcha.bin?key=" + key

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    print("캡차 이미지 저장")
    response_body = response.read()
    with open('captcha.jpg', 'wb') as f:
        f.write(response_body)
else:
    print("Error Code:" + rescode)

캡차 이미지 저장


In [17]:
#  사용자 입력값 검증 요청
import urllib.request

client_id = "DkiHrMnuqr5Rs9qsPs2f" # 개발자센터에서 발급받은 Client ID 값
client_secret = "LnZR3PAbKk" # 개발자센터에서 발급받은 Client Secret 값
code = "1"
key = "QKVQYCRj1xWmg1MC"
value = "9NHC8KA4"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code + "&key=" + key + "&value=" + value

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)


{"result":true,"responseTime":52.48}


* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [19]:
# 사용자 입력값 검증 요청
import os
from dotenv import load_dotenv
import requests

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET
key = "QKVQYCRj1xWmg1MC"
# os.getenv("NAVER_CAPTCHA_KEY")  # .env 파일의 NAVER_CAPTCHA_KEY

# 사용자 입력값
code = "1"
value = "9NHC8KA4"  # 실제 사용시 사용자 입력값으로 대체 필요

# URL 구성
# query string key1=value1&key2=value2
url = f"https://openapi.naver.com/v1/captcha/nkey?code={code}&key={key}&value={value}"

# 요청 헤더
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

# 요청 보내기
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # 4xx, 5xx 오류 발생시 예외 발생
    
    # 응답 처리
    print(response.text)
    
except requests.exceptions.RequestException as e:
    print(f"Error occurred: {e}")
    print(f"Status Code: {response.status_code if 'response' in locals() else 'N/A'}")


Error occurred: 403 Client Error: Forbidden for url: https://openapi.naver.com/v1/captcha/nkey?code=1&key=QKVQYCRj1xWmg1MC&value=9NHC8KA4
Status Code: 403


### 5. 블로그 검색하기

In [29]:
import requests
from pprint import pprint
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
}

# query string 문자열을 dict 선언
payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'date'
}

url = 'https://openapi.naver.com/v1/search/blog.json' #?query=파이썬&display=100&sort=sim

# requests get(url, params, headers) 요청 
res = requests.get(url, params=payload, headers=headers)
# json() 함수로 응답 결과 가져오기
items_data = res.json()['items']
print(type(res.json()))
#pprint(res.json())
print(len(items_data), type(items_data), type(items_data[0]))
pprint(items_data[0])

#[{},{},{}] 
# [] - 100 개의 데이터, {title:'',link:''} - 1개의 블로그 글 데이터

<class 'dict'>
100 <class 'list'> <class 'dict'>
{'bloggerlink': 'blog.naver.com/miri-campus',
 'bloggername': '미리캠퍼스',
 'description': '9 <b>파이썬</b>을 이용한 그림/영상 그리기와 편집 (6시간) 간단한 프로그래밍을 통해 그림이나 영상을 '
                '생성하고 편집 가능한 기능 실습 10 ChatGPT 활용 틀과 이를 활용한 게임 개발 실습 생성형 AI의 '
                '발전과 적용... ',
 'link': 'https://blog.naver.com/miri-campus/223944128778',
 'postdate': '20250723',
 'title': '공학 교구활용 안내'}


In [30]:

# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'
items_list = list()
item_list = []
for item in items_data:
#     print(item)
    item_list.append(item['title'])
    item_list.append(item['bloggername'])
    item_list.append(item['description'])
    item_list.append(item['bloggerlink'])
    item_list.append(item['link'])
    item_list.append(item['postdate'])

    items_list.append(item_list)
    item_list = []

print(items_list) #[[],[]]

# 'data/nhnblog.txt' 파일 생성하기
with open('data/nhnblog.txt','w',encoding="utf-8")as file:
    for items in items_list:
        for item in items:
            item = item + '\n'
            file.write(item)
        file.write('-'*150+'\n')

[['공학 교구활용 안내', '미리캠퍼스', '9 <b>파이썬</b>을 이용한 그림/영상 그리기와 편집 (6시간) 간단한 프로그래밍을 통해 그림이나 영상을 생성하고 편집 가능한 기능 실습 10 ChatGPT 활용 틀과 이를 활용한 게임 개발 실습 생성형 AI의 발전과 적용... ', 'blog.naver.com/miri-campus', 'https://blog.naver.com/miri-campus/223944128778', '20250723'], ['하루 목표 달성률 높이는 체크리스트 작성법', '도형레코드 DH record', "대신 '일주일 안에 <b>파이썬</b> 기본 문법 강의 3개 수강하고 예제 코드 따라 하기'와 같이 현실적인 목표를 설정하는 것이 바람직합니다. 목표 설정 시 현재 자신의 역량 수준을 객관적으로 평가하는 과정이... ", 'blog.naver.com/jigengs', 'https://blog.naver.com/jigengs/223911881591', '20250723'], ['2025.7.16 <b>파이썬</b>', 'kevinpanda0801님의 블로그', '', 'blog.naver.com/kevinpanda0801', 'https://blog.naver.com/kevinpanda0801/223944120302', '20250723'], ['spyder IDE로 python 사용하기', '세상을 가치있게', 'Spyder의 사용은 anacoda 를 설치하여 사용하는 것을 권장: Anaconda 프로그램을 설치하면, Python과 함께 Jupyter Notebook, NumPy, pandas, matplotlib 등 <b>파이썬</b>으로 데이터 사이언스를... ', 'blog.naver.com/gwihyeonseo', 'https://blog.naver.com/gwihyeonseo/223944119740', '20250723'], ['[python] pickle, copy, mask', '세상을 가치있게', 'pickle 모듈은